<img src="https://images.unsplash.com/photo-1466096115517-bceecbfb6fde?ixlib=rb-0.3.5&ixid=eyJhcHBfaWQiOjEyMDd9&s=427bcc1d8e2505d31a239d0de6b13f75&auto=format&fit=crop&w=1950&q=80"  width="900" height="400">

This excercise will help us to get started with **navie bayes generative classifier** and using tools(pandas, sklearn etc) from python eco system.

**Problem statement:** classify SMS messages as *HAM* or *SPAM* using **naive bayes** in supervised machine setting.
See this link to get an idea supervised learning workflow [supervsed learning workflow](http://www.allprogrammingtutorials.com/tutorials/introduction-to-machine-learning.php)

**Dataset:** We will use [SMS Spam Collection Data Set](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) from UCI machine learning repository.

credit:

- This notebook has taken some text processing idea from
https://radimrehurek.com/data_science_python/
- some of the images are from https://cdn.pixabay.com
- https://unsplash.com


Running this notebook with anaconda installation requires installing Textblob libray for text processing



# Running following commands to install Textblob 

Here is more infomation about [textblob](https://textblob.readthedocs.io/en/dev/)

In [ ]:
import os

In [ ]:
# output should be 0 after successful install
# run this only once. Comment later
os.system('conda install -c conda-forge textblob')

In [ ]:
#Must for inline plot
%matplotlib inline 
import requests
import numpy as np
import pprint # for pretty printing
import os # listing and managing file patho
import zipfile # for zip and unzip utilities
import pandas # for data analysis
import csv
import matplotlib.pyplot as plt # for plotting
from textblob import TextBlob
import seaborn as sb
sb.set()

In [ ]:
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
r = requests.get(data_url)
#r.content

Let's download and save the zip file

In [ ]:
sms_zip_file = 'smsspamcollection.zip'
#http = urllib3.PoolManager()
with open(sms_zip_file, 'wb') as out_file:
    out_file.write(r.content)

# Let's verify it. 
**make sure output of following command contains smsspamcollection.zip file**

In [ ]:
#Let verify it. See how you can run linux bash command using !
dir_listing = os.listdir('.') # list content of current directory
print(dir_listing)

# Q 1 (1 point): Can you complete following  code to check if sms_zip_file is present in above output. replace ? in following line with your code
# hint get the current directory content in a list and use the *in* operator to check list membership

In [ ]:

assert sms_zip_file ? , "directory doesn't contain {}".format(sms_zip_file) # hint look  in operator

In [ ]:
with zipfile.ZipFile(sms_zip_file,"r") as zip_ref:
    zip_ref.extractall("data")

# Let's list the content of the new data folder

In [ ]:
print(os.listdir('./data'))

SMSSpamCollection file contains around 5k SMS messages. Checkout readme file for details.

**Let's open this file and store line in python list**

In [ ]:
with  open('./data/SMSSpamCollection', 'r') as f:
    sms_messages = f.readlines()

In [ ]:
print(sms_messages[0:10]) # printing 10 messages

This is tab seperated(\t) file with a new line in the end. Let remove new line. Note that label and actual message **ham, spam** is seperated by tab.

In [ ]:
# Following code show how to write list comprehension. We could have done this using for loop too.
# [<some_func>(x) for x in <something> if  <some_condition_is_true>]
sms_messages = [m.rstrip() for m in sms_messages] # we are not using if condition part
print('Number of sms messages is {}'.format(len(sms_messages)))

# Let's check couple of messages again

In [ ]:
for idx, msg in enumerate(sms_messages[0:20]): # see how we can slice list using : operator
    print('message id {}  {}'.format(idx, msg))

**This is our  data set $\mathcal{D} = \{({x_i}, y_i)\}_{i=1}^{N=5574}$ $x_i$ is sms message and $y_i$ is label(ham or spam)**. Using  this we will train(learn parameters $\theta$ of a models(Naive bayes, Discriminant anlaysis based etc.)) and use trained model to classify new messages as ham or spam

First step before jumping into using any machine learning model is understanding the data by **describing it's statistical attribute and visualizating samples or sample property**.
We can use CSV file reader and try to accomplish above task. But as they say python is a language with **battery(libraries) included**. Let's use **pandas and matplotlib** libraries to do this task as cleanly as possible. What to describe and what to plot will be an essential skill we build as we do various data science or machine learning tasks. Also with time you will also built a knowledge of various packages available for different domain in python eco system. Most of the time reading blog and google search does the job of finding right libraries. Various packages for download and installation are avaiable at [PyPI - the Python Package Index](https://pypi.python.org/pypi)

**Optional**
This is [10 Minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html)

If you have more time look into this link [Pandas Tutorial: DataFrames in Python](https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python#gs.dEdNuDM)

In [ ]:
# You will see how wrapping the file in pandas simplify lot of tasks
messages = pandas.read_csv('./data/SMSSpamCollection', sep='\t', quoting=csv.QUOTE_NONE,
                           names=["label", "message"])
messages.head(6) # there are other functions like tail and sample to check record in dataset

# Let's try to understand various attribute of the data

*How many messages in each group etc.*

In [ ]:
messages.groupby('label').describe()

*How long are each messages*. See how we can attach a new column(pandas Series) to the pandas object.

It uses lambda(anonymous funtion) and map tell what to do with each entries in **message** column.
One can write a python function and pass it there too like

def get_length(msg):

    return len(msg)

**messages['length'] = messages['message'].map(get_length)**

But we will take more pythonic route

In [ ]:
messages['length'] = messages['message'].map(lambda text: len(text))

In [ ]:
messages.head()

Now we have length attribute.
**To see the whole picture. Let plot length distribution**

In [ ]:
messages.length.plot(bins=20, kind='hist')

Looks like there are enough messages of length upto 150 but very few messages are too long(>400).

Reading  exact values and properties from graph is hard.
Let's try to summarize this distribution(hist) of sms messages length

In [ ]:
messages.length.describe()

**max** value tell us there are messages  of length 910. What are these message?

In [ ]:
print('Longest message is {}'.format(list(messages.message[messages.length > 900])))

is there any difference in message length between spam and ham?

In [ ]:
messages.hist(column='length', by='label', bins=50)

See how length of ham messages is clustered rougly around [20 10] and spam messages length is clustered around 150.
**Looks like** on average spam messages has more length.

# Q2 (2 = 1+1 point): Can you write the code to summarize above per class disribution of messages length(**to be more precise about observation**). i.e. can you group messages and describe their length?

In [ ]:
# Write your answer here
# hint use groupby and select length and use describe

- As you can see  spam class has less number of example than ham class. This is called **class imbalance** issue. 
- We need to be carfeful in machine learning application about class imbalance. There are way to handle it but for current exercise let's ignore it

# Q3(1 point). Can you suggest some way to handle class imbalance issue. This is an open question. So don't worry about  being right or wrong. I am looking for some fresh thoughts

*write you answer in this cell (doble click and it will be editable)*

## Computer only understand scalar or vector or matrices. We need to convert text to vectors(features).


# Feature engineering
(It was a big  sought after skill before deep learning at least in vision applications)

we'll use the [Bag-of-words model](https://en.wikipedia.org/wiki/Bag-of-words_model) approach for creating feature
representing our sms messages.

### Bog of word model for document:

In BOG  we treat document as collection of word without any order, like they are lying in bag. 

Two model to represent sms/document in a vector form are:

- **Bernoulli document model: mes**sage is represented by a binary feature vector of absence or presence of word.
- **Multinomial document model**: message is represented by an integer feature vector of word frequency.

 Later we will see
there are better model for sentence or document representation **where words order matters**. There are model which takes into account the word order like [N-gram](https://en.wikipedia.org/wiki/N-gram) and importance of words across documents(bags) [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) etc.
Infact Deep learning has enabled us to learn better embedding of words using context of words(co occurance).
We will try to use them in **deep learning section** [**optional** see widely used [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/)] now a days.

We will use **Multinomial document model** in this exercise

Converting to vector is bit involved and require a good understanding of NLP(natural language processing).

But as we can imagine to convert a message into vector we need to
1. convert a sentence into word token
2. Normalize the words i.e do we care about Capital form(Cow vs cow), inflected form ("goes" vs. "go")
3. Build a dictionary of words and map the messages into vector using this dictionary
4. Finally train a  Naive Bayes model

**Again we will use a python library [Textblob](http://textblob.readthedocs.io/en/dev/quickstart.html) to do heavy lifting for us.**

write a function that will split a message into its individual words

In [ ]:
def split_into_tokens(message):
    #message = unicode(message, 'utf8')  # convert bytes into proper unicode
    return TextBlob(message).words

Here are some of the original texts again:

In [ ]:
messages.message.head()

same messages, tokenized 

In [ ]:
messages.message.head().apply(split_into_tokens)

With textblob, normalize words into their base form [lemmas](https://en.wikipedia.org/wiki/Lemmatisation) with

In [ ]:
import string
def split_into_lemmas(message):
    message = message.lower()
    words = TextBlob(message).words
    no_punc = []
    # for each word, take its "base form" = lemma 
    words =  [word.lemma for word in words]
    for word in words:
        line = "".join(char for char in word if char not in string.punctuation)
        no_punc.append(line)
    return no_punc    
        

# see how head portion changes
preprocessed_messages =messages.message.apply(split_into_lemmas)

In [ ]:
preprocessed_messages.head()

In [ ]:
messages_labels = messages.label
messages_labels.head()

In [ ]:
preprocessed_messages.shape

See how **goes** is changed to **go**. We can do some more pre processing  with some justification but let's go
ahead with current manipulation.

There are lots of other natural language processing libraries in python for performing above activities like
- [Spacy](https://spacy.io/)
- [nltk](https://www.nltk.org/)

- One of the fundamental concern in machine learning is *Generalization*, **how well our machine is going to work well on unseen/future data. Does it generalize well on future data**?
- If we wanted to do well on given data, why  would we even bother to build an algorithm. We can just store the data and do a lookup for any sms message.

We will come back to this question later in the course when we talk about **algorithm/model selection and evaluation**.

One simple way to answer above question is to hide some portion of dataset and use remaining dataset for building the model i.e. learning the parameters. Once we have build the model, we can report some number/measure on hidden dataset to tell how well the model will perform on unseen data.

# Let's partition our data into training and test set

In [ ]:
training_set_portion =.9 # keep 90 % data for training
#LEt's create some random integer index and partition the data
number_of_examples = preprocessed_messages.shape[0]
print('Total examples are {}'.format(number_of_examples))
np.random.seed(0) # to make sure multiple run give same result
random_index = np.random.permutation(range(number_of_examples))
training_set_size = int(number_of_examples*training_set_portion)
print('train set size is {} test set size is {}'.format(training_set_size,number_of_examples - training_set_size))

In [ ]:
training_messages = preprocessed_messages[random_index[:training_set_size]]
training_labels = messages_labels[random_index[:training_set_size]]
test_messages = preprocessed_messages[random_index[training_set_size:]]
test_labels = messages_labels[random_index[training_set_size:]]

print('Shape of training X {} and train Y {}'.format(training_messages.shape, training_labels.shape))
print('Shape of test X {} and test Y {}'.format(test_messages.shape, test_labels.shape))

**We'll use training messages only for building the model**

- We need to convert each message into count vector. Where a ham/spam message is mapped to vector representing each word frequency in the message
    + To do this we need to build a dictionary of words first

# Q 4(1 point) Write code to update the python set. We are using set as it takes care of duplicate words.

In [ ]:
# How many unique words are in our dictionary
unique_word = set()
for message in training_messages:
    # write your code here to update unique_word set

In [ ]:
# how many words in vocabulary(V)
len(unique_word)

We will encode each message into len(unique_word) dimensional vector. In meachine learning we call acitivity like this feature engineering.

In [ ]:
#let' use default dictionary to assign each word a unique location in feature vector
from collections import defaultdict, Counter
word_to_index_dict = defaultdict(int)
for index , word in enumerate(unique_word):
    word_to_index_dict[word] = index

In [ ]:
# Let's create a reverse dictionary  for mapping index to word. It will help in debugging etc.
# See how we used dictionary comprehension
index_to_word_dict = { value:key  for key, value in word_to_index_dict.items()}

In [ ]:
print(training_messages.shape)

## we will convert each  messages into |V| dimensional vector, where |V| is size of our dictionary

## Let's create a numpy integer matrix of right size, initialized with zero

In [ ]:
# each row in training_X is our x_i
training_X = np.zeros((len(training_messages), len(unique_word)), dtype=int)
print(training_X.shape)

# Q5 (1 point) use Counter from collections to count frequency of each word in a message. I have written rest of the code for updating frequency into right index of the feature vector.

In [ ]:
# Let's go over each training message, count the words using Counter and set count in feature vector for sms 

for sms_no, sms in enumerate(training_messages):
    word_freq =  # write your code here
    # setting the word count in sms_no row of sms_features
    for word, freq in word_freq.items():
        index_of_word = word_to_index_dict[word]
        training_X[sms_no][index_of_word] = freq
    

## Writing code is easy :)
## But how we check if it is correct
## Let's do some primitive checking on a sms message

In [ ]:
sms_no =3
message_word_count = Counter(training_messages.iloc[sms_no])
print(message_word_count)

# Let' check non zero location in sms_features to see if count is set properly
print('##Encoding for sms no {} in feature vector is ##'.format(sms_no))
for i, count in enumerate(training_X[sms_no]):
    if count >0:
        print(index_to_word_dict[i], count)


<font color= 'red' > Make sure Counter and encoding gives same results in above cell </font>

# We have successfully converted sms message into feature vector and
# collected them in numpy matrix

<img src="https://images.unsplash.com/photo-1522098543979-ffc7f79a56c4?ixlib=rb-0.3.5&ixid=eyJhcHBfaWQiOjEyMDd9&s=3deb7fa95bb0a7343a38b724cbee4b5a&auto=format&fit=crop&w=1868&q=80" alt="Well done" width="500" height="400">


# Let's convert ham and spam label to 1 and 0  respectively

In [ ]:
training_labels.tail(7)# can check from head too

In [ ]:
# This is our training_y value
training_y = (training_labels.values == 'ham').astype(int)

In [ ]:
# Let's check some lable value
training_y[-7:]


# Training the model or estimating parameters $\theta$ of the model
Now we have vector feature representation $x_i$ of our sms samples. 

Let review some theory and see what parameters we need to estimate for Naive bayes model.

We know that we classify a sms $x_i$  to a class c= HAM or c= SPAM which has maximum value of $P(c|x_i).$ Using bayes rule we have $P(c|x_i) = \frac{P(x_i|c) P(c)}{P(x_i)} \propto P(x_i|c) P(c)$ as normalization doesn't depend on class label. 

In naive bayes assumption for modelling class conditional densities we have $P(x_i|c) = \prod_j^D P(x_{ij}|c)$ assuming  $x_i \in \mathbb{R}^D$ i.e each example has $D$ dimentional features.

**Note:$D$ is size of our vacabulary ($|V|$) build from sms document corpus i.e D = |V|**

**what probability distribution we should choose for $P(x_{ij}|c)?$ **

Each value $x_{ij}$ is an integer values(count of words) and there are total $D$ different unique values(word). This definetly suits a **$D$ side die** situation. In our case die has $D = |V|$ sides= size of feature vector.

**Infact once we have learned $P(x_{ij}|c)?$ i.e probabilites of different sides for ham and spam die,**
** ham or spam sms generation in bag of word model is nothing but rolling ham or spam die. Pick the word dictated by the side of die throw.**

Now we  know that we can put multinomial distribution for such situation. Hence
<font size = 6> 
$P(x_i|c) = \frac{n !}{\prod_j^D x_{ij !}} P(c) \prod^{D} P(w_j|c)^{x_{ij}} \propto P(c) \prod^{D} P(w_j|c)^{x_{ij}}$ 
</font>
as normalization doesn't depend on class label

We know that using MLE estimate we have
<font size = 8> 
$P(w_j|c) = \frac{\sum_{i=1}^N x_{ij}\mathbb{1}(y_i=c)}{\sum_{k=1}^{D} \sum_{i=1}^N x_{ik}\mathbb{1}(y_i=c)}.$ 
</font>
where $\mathbb{1}$ is indicator function.


- Hence the parameters are nothing  nothing but relative frequency of $w_j$ in documents of class c=SPAM or c= HAM
with respect to the total number of words in documents of that class.

- We can sum our numpy sms_feature matrix along row or dim 0 to get total frequency of each feature for ham and spam class
- normalize total frequency of each feature with total frequency of all the features for each class.
- prior class  densites are estimated as $P(c) = \frac{N_c}{N}.$ Where $N_c$ are numer of document in class k.



# Let's learn the parameters for c= ham(1) and c= spam(0)



# For ham class

In [ ]:
# First estimate for ham

# summing up per feature count
training_X_ham = training_X[training_y ==1]
print(training_X_ham.shape)
per_feature_count =np.sum(training_X_ham, axis = 0)
per_feature_count.shape

np.count_nonzero(per_feature_count)
parameters_w_ham = per_feature_count/(np.sum(per_feature_count))

parameters_w_ham

# Let's estimate parameters for spam

# Q 6(1 Point): Write code similary to ham class to estimate following parameter parameters_w_spam for spam class

In [ ]:
# write your code here


print(parameters_w_spam)

# Zero probability issue
As we can see some of the probablity can be zero. It will create problem when we estimate probability of a new document in test set if that word was not in training set. 

If any of the term in product is zero it will result in zero product. If any of the class don't have this term then probability of this document for any class will be zero. It is an ambiguous situation. If we play log trick for comparing product of probability, we will be in troble as log of zero is not defined too.

One way to handle this situtation to add a fake 1 count of the word in each class. This is called Laplace law of sccession or add one smoothing.

We estimate
<font size = 8> 
$P(w_j|c) = \frac{\sum_{i=1}^N x_{ij}\mathbb{1}(y_i=c) + 1}{\sum_{k=1}^{D} \sum_{i=1}^N x_{ik}\mathbb{1}(y_i=c) + |V|}.$ 
</font>
where $\mathbb{1}$ is indicator function and $|V|$ is size of our dictionary.

This can be done by adding a row of ones to training_X_ham and training_X_spam


# New parameters Laplace law of sccession or add one smoothing.

# Q 7 (2 point) Estimate new parameter for ham class

In [ ]:
# Write your code here

print(parameters_w_ham)

# Q 8(1 point). Estimate new parameter for spam class 

In [ ]:
# Write your code here

print(parameters_w_spam)

# class probabilities

# Q 9 (1 point) estimate class probabilities P(c=ham) and P(c=spam)

In [ ]:
ham = # Write your code here for estimating ham class probability
spam = 1- ham
ham,spam

# Now we have learned the model(i.e its parameters, probabilities of different words occuring in ham dice and spam dice)

# How good is our model ?
- Let take out our test data and convert to count feature vector using same dictionary
- Calulate the probability if test data belonging to Ham or spam. i.e if probability if >=.5 Ham otherwise spam
 or we can calulate the ratio
 <font size = 5>
 $\frac{P(x_{test}|c=ham)}{P(x_{test}|c=spam)} = \frac{ P(c=ham)  \prod^{D}_{j =1} P(w_j|c=ham)^{x_{test,j}}} { P(c= spam)\prod^{D}_{j=1} P(w_j|c=spam)^{x_{test, j}}}$ 
 </font>
 
 **Note:Generally such large product of probabilties, turns out to be zero because of computer representation limits of real numbers.**
 
 Another option is let take log on right hand side and after some manipulation one can show that if
 <font size = 5>
  $\sum_{j =1}^{D} (x_{test,j})log (P(w_j|c=ham)) +log(P(c= ham)) \ge log(P(c=spam))+ \sum_{j =1}^{D} (x_{test,j})log (P(w_j|c=spam))$
  
  </font>
  
 then it is ham otherwise spam
 
 
 
 
- Compare our prediction of label with test data label and let's report accuracy

Let's do these steps

In [ ]:
test_X = np.zeros((len(test_messages), len(unique_word)), dtype=int)
print(test_X.shape)
len(word_to_index_dict)

In [ ]:
def build_feature(sms, word_to_index_dict):
    feature = np.zeros((len(word_to_index_dict),), dtype=int)
    word_freq =  Counter(sms)
    # setting the word count in sms_no row of sms_features
    for word, freq in word_freq.items():
        if word in word_to_index_dict:
            index_of_word = word_to_index_dict[word]
            feature[index_of_word] = freq
    return feature        
    
for sms_no, sms in enumerate(test_messages):
    test_X[sms_no] = build_feature(sms, word_to_index_dict)


# Again checking feature creation/encoding

In [ ]:
sms_no =2
message_word_count = Counter(test_messages.iloc[sms_no])
print(message_word_count)

# Let' check non zero location in sms_features to see if count is set properly
print('##Encoding for sms no {} in feature vector is ##'.format(sms_no))
for i, count in enumerate(test_X[sms_no]):
    if count >0:
        print(index_to_word_dict[i], count)


## We need test_y  value to compare predicted labels

# Q 10 (1 point) convert ham and spam to 1 and 0 integer as done in training set

In [ ]:
# This is our test_y value
test_y = # Write your code here


# Finally let's calculate ham/spam probability for test messages

In [ ]:
ham_score = np.zeros_like(test_y,dtype=float)
spam_score = np.zeros_like(test_y,dtype=float)
ham_score.shape, spam_score.shape# just printing to make sure shape is right

In [ ]:
def calculate_score(parameters,test_sms, class_prior):
    return np.sum(np.log(np.power(parameters,test_sms))) + class_prior

for idx, test_sms in enumerate(test_X):# this will fetch row by row, encoded test messages
    ham_score[idx] = calculate_score(parameters_w_ham,test_sms, np.log(ham))
    spam_score[idx] = calculate_score(parameters_w_spam, test_sms, np.log(spam))

    

In [ ]:
# Let print some values for visual comparision/verification
ham_score[0:2], spam_score[0:2], test_y[0:2]

In [ ]:
# predict the label ham(1) or spam(0)
ham_or_spam = (ham_score >= spam_score).astype(int)

In [ ]:
ham_or_spam[0:5], test_y[0:5]

# Accuracy calculation

# Q 11(1 point). Write code to calculate accuracy. Accuracy is defined as average number of correct predicted label

In [ ]:
accuracy = # Write your code here

print('accuracy on test set is {}'.format(accuracy))

In [ ]:
def predict_ham_or_spam(message):
    feature = build_feature(message, word_to_index_dict)
    ham_score = calculate_score(parameters_w_ham,feature, np.log(ham))
    spam_score = calculate_score(parameters_w_spam, feature, np.log(spam))
    
    return 'ham' if ham_score > spam_score else 'spam'
    
    


# Let's see how it works on new spam message which is a modified  training message

In [ ]:
predict_ham_or_spam(' your mailbox messaging sm alert call back 09056242159 to retrieve your message'.split())

<font color = 'BlueViolet' size = 6> Try some messages with same distribution as training messages to see how well it does </font>

<font color = 'BlueViolet' size = 6> Following code will show why python eco system shines.
We will use python library sklearn to build  multinomial Naive Bayes classifier </font>

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Q 12 (2 = 1 +1 point) Write code here to train MultinomialNB classifer and it accuracy in test set
Your code should not be more than 3 lines. Check if you get same accuracy number

In [ ]:
# write your code here